In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
fn = '/content/drive/MyDrive/Polynomial/Cell_Phones_and_Accessories.json.gz'
print(os.path.isfile(fn))
df = pd.read_json(fn, lines=True, compression='gzip', nrows=3200000)
df.tail()

True


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
3199995,3,True,"07 23, 2014",A2OCQU4VYUUSD6,B00KAM1E7G,Briuana Wright,NaN,Three Stars,1406073600,NaN,NaN,{'Color:': ' Mint'}
3199996,5,True,"07 23, 2014",AGQK8QOJNRHQU,B00KAM1E7G,Jessi,Love it it is super cute and easy to carry.,Five Stars,1406073600,NaN,NaN,{'Color:': ' Mint'}
3199997,5,True,"07 23, 2014",AOWING33U8L2T,B00KAM1E7G,diver,Perfect fit. Great protection.,Five Stars,1406073600,NaN,NaN,{'Color:': ' Purple'}
3199998,5,False,"07 18, 2014",A3JW9UNR8ZZV1B,B00KAM1E7G,Paz,Got this case for my sisters Droid Ultra. Fits...,Fits perfect. She absolutley loves it,1405641600,NaN,NaN,{'Color:': ' Purple'}
3199999,5,True,"07 16, 2014",A1XIPPCV1ZERHL,B00KAM1E7G,Blitz Krieg,Bought for the wife's new Droid Maxx. Perfect...,"Great fit, good protection at a low price",1405468800,NaN,NaN,{'Color:': ' Mint'}


In [4]:
df = pd.DataFrame([df.overall, df.reviewText , df.summary]).transpose()
df.dropna(inplace=True)
df.describe()

,overall,reviewText,summary
count,3197385,3197385,3197385
unique,5,2872555,1391793
top,5,Good,Five Stars
freq,1755964,8579,463348


In [5]:
df.head()

,overall,reviewText,summary
0,5,If your into space this is the Calendar for you.,Five Stars
1,5,Awesome pictures!,Five Stars
2,5,Great wall art and information for space explo...,Five Stars
3,5,"As always, it is a quality calendar full of ve...",I love it. I buy a new one every year
4,5,This is a fantastic calendar. This is my third...,Great Calendar.


In [6]:
df['overall'].replace(to_replace=[1,2],value='negative',inplace=True)
df['overall'].replace(to_replace=[3],value='neutral',inplace=True)
df['overall'].replace(to_replace=[4,5],value='positive',inplace=True)

In [7]:
df.tail(10)

,overall,reviewText,summary
3199989,positive,"I really like it, the only thing is that the c...","I really like it, the only thing is that the c..."
3199990,positive,Great,Five Stars
3199991,neutral,I have to take out the phone to charge it. The...,There are pockets inside for an ID card which ...
3199992,positive,Excellent quality great slim fit. Very durable,Five Stars
3199993,positive,"This is a great phone case, I would recommend ...",I love it!
3199994,positive,a little flimsy but a neat little casement. Ph...,Great for taking on vacation or shopping
3199996,positive,Love it it is super cute and easy to carry.,Five Stars
3199997,positive,Perfect fit. Great protection.,Five Stars
3199998,positive,Got this case for my sisters Droid Ultra. Fits...,Fits perfect. She absolutley loves it
3199999,positive,Bought for the wife's new Droid Maxx. Perfect...,"Great fit, good protection at a low price"


In [8]:
X = '<r>' + df['reviewText']+'<s>' + df['summary']
y = df['overall']

In [9]:
import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text
    
X = X.apply(clean_text)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

Linear SVC -: 

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer(token_pattern=r'\w{2,}')),
                     ('clf', LinearSVC(random_state=1)),
])

text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(token_pattern='\\w{2,}')),
                ('clf', LinearSVC(random_state=1))])

In [19]:
predictions = text_clf.predict(X_test)

In [20]:
unique, counts = np.unique(predictions, return_counts=True)
print(np.asarray((unique, counts)).T)

[['negative' 163073]
 ['neutral' 32562]
 ['positive' 603712]]


In [21]:
# Print the overall accuracy
import sklearn.metrics as metrics 
from sklearn.metrics import accuracy_score
print(metrics.accuracy_score(y_test,predictions)*100)

89.14689114990111


Linear Support Vector Machine

In [23]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

sgd = Pipeline([('tfidf', TfidfVectorizer(token_pattern=r'\w{2,}')),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))


accuracy 0.7622484352853016
89.14689114990111


XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
Xgb = Pipeline([('tfidf', TfidfVectorizer(token_pattern=r'\w{2,}')),
                ('clf', XGBClassifier(max_depth=200, n_estimators=400, subsample=1, learning_rate=0.07, reg_lambda=0.1, reg_alpha=0.1,\
                       gamma=1)),
               ])
Xgb.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

In [ ]:
import pickle
pickle.dump(text_clf,open("modelSVC.pkl","wb"))